In [26]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
from tqdm import tqdm
import sqlite3

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [20]:
with open('../datafiles/pickles/affr_bow_500','rb') as affr_bow_500:
    affr_bow_500 = pickle.load(affr_bow_500).toarray()
    print(affr_bow_500.shape,affr_bow_500[0])
    print('----------------------------------------------------------------------------------------------------------------------------------------------')

with open('../datafiles/pickles/affr_bigram_bow_500','rb') as affr_bigram_bow_500:
    affr_bigram_bow_500 = pickle.load(affr_bigram_bow_500).toarray()
    print(affr_bigram_bow_500.shape,affr_bigram_bow_500[0])
    print('----------------------------------------------------------------------------------------------------------------------------------------------')

with open('../datafiles/pickles/affr_ngram_tfidf_500','rb') as affr_ngram_tfidf_500:
    affr_ngram_tfidf_500 = pickle.load(affr_ngram_tfidf_500).toarray()
    print(affr_ngram_tfidf_500.shape,affr_ngram_tfidf_500[0])
    print('----------------------------------------------------------------------------------------------------------------------------------------------')

with open('../datafiles/pickles/avg_w2v','rb') as avg_w2v:
    avg_w2v = np.array(pickle.load(avg_w2v))
    print(avg_w2v.shape,avg_w2v[0])
    print('----------------------------------------------------------------------------------------------------------------------------------------------')

with open('../datafiles/pickles/tdifd_weighted_w2v','rb') as tdifd_weighted_w2v:
    tdifd_weighted_w2v = np.array(pickle.load(tdifd_weighted_w2v))
    print(tdifd_weighted_w2v.shape,tdifd_weighted_w2v[0])

(364171, 500) [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 2 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 2 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 

In [30]:
conn = sqlite3.connect('../datafiles/amazon_reviews.sqlite')
data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score!=3""",conn)
def scr(s):
    if(s>3):
        return 1
    else:
        return 0
data['Score'] = data['Score'].apply(scr)
cus_data = data.drop_duplicates(subset={'UserId','ProfileName', 'Time', 'Text'},keep='first')
cus_data = cus_data[cus_data['HelpfulnessNumerator']<=cus_data['HelpfulnessDenominator']]
Scr = cus_data.Score

In [31]:
class Knn:
    def __init__(self,X,Y,time_splits=10,algorithm='auto'):
        self.X = X
        self.Y = Y
        self.time_splits = time_splits
        self.algorithm = algorithm
        
    def KNN_TS(self):
        X_train_n_cv, X_test, Y_train_n_cv, Y_test = train_test_split(self.X,self.Y,test_size=0.3)
        tscv = TimeSeriesSplit()
        TimeSeriesSplit(max_train_size=None, n_splits=self.time_splits)
        scrs = list()
        for n in tqdm(range(1,30)):
            cv_score = list()
            train_score = list()
            for train_index, cv_index in tscv.split(X_train_n_cv):
                X_train,Y_train = X_train_n_cv[train_index],Y_train_n_cv[train_index]
                X_cv,Y_cv = X_train_n_cv[cv_index],Y_train_n_cv[cv_index]
                model = KNeighborsClassifier(n_neighbors = n, algorithm = self.algorithm)  
                model.fit(X_train,Y_train)
